In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

import json
import os
import pandas as pd
pd.set_option("display.max_columns", 30)
import requests


In [2]:
""" 
1.  get the data from S3
2. weather data tranformations 
3. taxi trips transformations   - DONE
4. update payment_type_master   - DONE
5. updat company_master         - DONE
6. update taxi_trips with company and payment_type id's (replace the string values with id's from the latest master pables) -DONE
7. upload the weather data to s3
8. upload taxi data to s3
9. upload the newest payment_type_master and company_master


"""

" \n1.  get the data from S3\n2. weather data tranformations \n3. taxi trips transformations   - DONE\n4. update payment_type_master   - DONE\n5. updat company_master         - DONE\n6. update taxi_trips with company and payment_type id's (replace the string values with id's from the latest master pables)\n7. upload the weather data to s3\n8. upload taxi data to s3\n9. upload the newest payment_type_master and company_master\n\n\n"

#### taxi_trips tranformation codes

In [3]:
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d") 

url = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp between '{formatted_datetime}T00:00:00' and '{formatted_datetime}T23:59:59'"

response = requests.get(url)

data = response.json()

In [4]:
taxi_trips = pd.DataFrame(data)
taxi_trips

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_community_area,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
0,82878f6794e487ca17b7d084c4b035cefeb88d12,cf5f0f5970c4961f96199dfcb1b11c9835a7fa2db48ff0...,2024-03-13T23:45:00.000,2024-03-14T00:00:00.000,869,11.23,22,28.75,0,0,0,28.75,Prcard,City Service,41.92276062,-87.699155343,"{'type': 'Point', 'coordinates': [-87.69915534...",NaN,NaN,NaN,NaN,NaN,NaN
1,4914bc18854fa2e923cbc848c41f64edf913b52c,e88fa53c774c82956a2aad60dcbdd260f05b157cf197da...,2024-03-13T23:45:00.000,2024-03-14T00:00:00.000,1130,17.23,76,42,9.3,0,4,55.8,Credit Card,Blue Ribbon Taxi Association,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",8,41.899602111,-87.633308037,"{'type': 'Point', 'coordinates': [-87.63330803...",NaN,NaN
2,9e75abb951660293deb8f726e304a9caecf187ea,96c3fa383c9adacc965af778248d6ef1b3a20ada7e623e...,2024-03-13T23:45:00.000,2024-03-14T00:15:00.000,2025,26.43,76,63,13.9,0,6,83.4,Mobile,Flash Cab,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",41,41.794090253,-87.592310855,"{'type': 'Point', 'coordinates': [-87.59231085...",NaN,NaN
3,dc178c25a88693532447ad42d534a57d9395b18d,b89de8fefce2927a1bbc747ac861743fc5482ed36590d2...,2024-03-13T23:45:00.000,2024-03-14T00:00:00.000,938,7.93,76,22,3.98,0,4,30.48,Mobile,Taxicab Insurance Agency Llc,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",11,41.978829526,-87.771166703,"{'type': 'Point', 'coordinates': [-87.77116670...",NaN,NaN
4,f33b226c200ea37f1b2e3f38b42f9aa908666086,b12c6be9f088bab50b80d41139e9c64c78030e062ad567...,2024-03-13T23:45:00.000,2024-03-13T23:45:00.000,360,0.9,32,6,4,0,0,10,Credit Card,Taxi Affiliation Services,41.878865584,-87.625192142,"{'type': 'Point', 'coordinates': [-87.62519214...",28,41.874005383,-87.66351755,"{'type': 'Point', 'coordinates': [-87.66351754...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,d88c02968862eff8680413cf2ddf89c2e91d8a22,3497d2b935bf09123c257fd1ff03b4ca266e067fac8e97...,2024-03-13T21:45:00.000,2024-03-13T22:15:00.000,1500,17.1,76,42,9.3,0,4,55.3,Credit Card,Taxi Affiliation Services,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",28,41.874005383,-87.66351755,"{'type': 'Point', 'coordinates': [-87.66351754...",NaN,NaN
996,da48752ceaad13c6f1f6bce07b4eda88c09a9bb9,acad560bbc140c4015f4685c6559c93b61ccaf0f7d8014...,2024-03-13T21:45:00.000,2024-03-13T22:15:00.000,1890,15.58,76,39.5,11.25,0,5,56.25,Credit Card,Taxicab Insurance Agency Llc,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",6,41.944226601,-87.655998182,"{'type': 'Point', 'coordinates': [-87.65599818...",NaN,NaN
997,d7cb5039389f5c99598a955090a2144924608336,55e53aceec1519445bae3e8414b4cbf11e59e50800f714...,2024-03-13T21:45:00.000,2024-03-13T22:15:00.000,1297,12.95,NaN,33.75,7.65,0,4,45.9,Mobile,Blue Ribbon Taxi Association,NaN,NaN,NaN,8,41.898331794,-87.620762865,"{'type': 'Point', 'coordinates': [-87.62076286...",NaN,17031081300
998,e9a4beeb363454b3893e3522eb3701a93fd80cf7,8ef1056519939d511d24008e394f83e925d2539d668a00...,2024-03-13T21:45:00.000,2024-03-13T22:00:00.000,659,0,24,9.9,2.35,0,0,12.25,Mobile,5 Star Taxi,41.901206994,-87.676355989,"{'type': 'Point', 'coordinates': [-87.67635598...",24,41.901206994,-87.676355989,"{'type': 'Point', 'coordinates': [-87.67635598...",NaN,NaN


In [5]:
taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract"], axis = 1, inplace =True)
taxi_trips.drop(["pickup_centroid_location", "dropoff_centroid_location"], axis = 1, inplace =True)

taxi_trips.dropna(inplace=True)

taxi_trips.rename(columns={"pickup_community_area" : "pickup_community_area_id",
                           "dropoff_community_area": "dropoff_community_area_id"}, inplace = True)

taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("h")
#taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_end_timestamp"])



In [6]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_community_area_id,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
1,4914bc18854fa2e923cbc848c41f64edf913b52c,e88fa53c774c82956a2aad60dcbdd260f05b157cf197da...,2024-03-13T23:45:00.000,2024-03-14T00:00:00.000,1130,17.23,76,42,9.3,0,4,55.8,Credit Card,Blue Ribbon Taxi Association,41.980264315,-87.913624596,8,41.899602111,-87.633308037,2024-03-13 23:00:00
2,9e75abb951660293deb8f726e304a9caecf187ea,96c3fa383c9adacc965af778248d6ef1b3a20ada7e623e...,2024-03-13T23:45:00.000,2024-03-14T00:15:00.000,2025,26.43,76,63,13.9,0,6,83.4,Mobile,Flash Cab,41.980264315,-87.913624596,41,41.794090253,-87.592310855,2024-03-13 23:00:00
3,dc178c25a88693532447ad42d534a57d9395b18d,b89de8fefce2927a1bbc747ac861743fc5482ed36590d2...,2024-03-13T23:45:00.000,2024-03-14T00:00:00.000,938,7.93,76,22,3.98,0,4,30.48,Mobile,Taxicab Insurance Agency Llc,41.980264315,-87.913624596,11,41.978829526,-87.771166703,2024-03-13 23:00:00
4,f33b226c200ea37f1b2e3f38b42f9aa908666086,b12c6be9f088bab50b80d41139e9c64c78030e062ad567...,2024-03-13T23:45:00.000,2024-03-13T23:45:00.000,360,0.9,32,6,4,0,0,10,Credit Card,Taxi Affiliation Services,41.878865584,-87.625192142,28,41.874005383,-87.66351755,2024-03-13 23:00:00
5,742f24b4519c8a3aeaf725a58eec582c0102bd0c,db39bc53c7204111f0786edff94251c636dcce5fec8dbe...,2024-03-13T23:45:00.000,2024-03-14T00:00:00.000,1080,6.3,28,18.75,3.85,0,0,22.6,Credit Card,Taxi Affiliation Services,41.874005383,-87.66351755,21,41.938666196,-87.711210593,2024-03-13 23:00:00


### taxi_trips tranformation function

In [7]:
def taxi_trips_transformations(taxi_trips: pd.DataFrame)->pd.DataFrame:
    """Perform transformations with the taxi data.

    Args:
        taxi_trips (pd.DataFrame): 
        The DataFrame holding the daily taxi trips.

    Returns:
        pd.DataFrame: 
        The cleaned, transformed DataFrame holding the daily taxi trips.
    """

    if not isinstance(taxi_trips, pd.DataFrame):
        raise TypeError("taxi_trips is not a valid pandas DataFrame.")

    taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract", 
                     "pickup_centroid_location","dropoff_centroid_location"], axis = 1, inplace =True)

    taxi_trips.dropna(inplace=True)

    taxi_trips.rename(columns={"pickup_community_area" : "pickup_community_area_id",
                            "dropoff_community_area": "dropoff_community_area_id"}, inplace = True)

    taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("h")
    #taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_end_timestamp"])

    return taxi_trips


#### company update codes

In [8]:
company_master = taxi_trips["company"].drop_duplicates().reset_index(drop = True)
company_master = pd.DataFrame(
    {
        "company_id": range(1,len(company_master)+1),
        "company": company_master
    }
) 
company_master.tail()

,company_id,company
16,17,Tac - Yellow Cab Association
17,18,Top Cab
18,19,Patriot Taxi Dba Peace Taxi Associat
19,20,Tac - Checker Cab Dispatch
20,21,3591 - 63480 Chuks Cab


In [9]:
new_company_data = [
    {"company": "Choice Taxi Association Inc"},
    {"company": "X"},
    {"company": "Y"},
]
new_company_mapping = pd.DataFrame(new_company_data)
new_company_mapping

,company
0,Choice Taxi Association Inc
1,X
2,Y


In [10]:
company_max_id = company_master["company_id"].max()
company_max_id

21

In [11]:
new_companies_list = []

for company in new_company_mapping["company"].values:
    if company not in company_master["company"].values:
        new_companies_list.append(company)

#one line part
new_companies_list= [company for company in new_company_mapping["company"].values if company not in company_master["company"].values]
new_companies_list

['X', 'Y']

In [12]:
new_companies_df = pd.DataFrame({
    "company_id": range(company_max_id +1, company_max_id++ len(new_companies_list)+1),
    "company": new_companies_list
})

new_companies_df

,company_id,company
0,22,X
1,23,Y


In [13]:
updated_company_master = pd.concat([company_master,new_companies_df], ignore_index=True)
updated_company_master.tail()

,company_id,company
18,19,Patriot Taxi Dba Peace Taxi Associat
19,20,Tac - Checker Cab Dispatch
20,21,3591 - 63480 Chuks Cab
21,22,X
22,23,Y


In [14]:
def update_company_master(taxi_trips: pd.DataFrame, company_master: pd.DataFrame)->pd.DataFrame:
    """Extend the company master with new companies if there are new companies.

    Args:
        taxi_trips (pd.DataFrame): 
        DataFrame holding the daily taxi trips.
        company_master (pd.DataFrame): 
        DataFrame holding the company_master data.

    Returns:
        pd.DataFrame: 
        The updated company_master data, if new companies are in the taxi data, they will be loaded to it.
    """

    company_max_id = company_master["company_id"].max()
    new_companies_list= [company for company in taxi_trips["company"].values if company not in company_master["company"].values]
    new_companies_df = pd.DataFrame({
        "company_id": range(company_max_id +1, company_max_id + len(new_companies_list)+1),
        "company": new_companies_list
    })

    new_companies_df
        
    updated_company_master = pd.concat([company_master,new_companies_df], ignore_index=True)
    return updated_company_master

In [15]:
taxi_trips_company_only = pd.DataFrame({
    "company_id":[1,2,3],
    "company": ["X","Y","Koam Taxi Association"]
})

taxi_trips_company_only

,company_id,company
0,1,X
1,2,Y
2,3,Koam Taxi Association


In [16]:
updated_company_master = update_company_master(taxi_trips=taxi_trips_company_only, company_master=company_master)

In [17]:
updated_company_master.tail()

,company_id,company
19,20,Tac - Checker Cab Dispatch
20,21,3591 - 63480 Chuks Cab
21,22,X
22,23,Y
23,24,Koam Taxi Association


### payment_type_master codes

In [18]:
payment_type_master = taxi_trips["payment_type"].drop_duplicates().reset_index(drop = True)
payment_type_master = pd.DataFrame(
    {
        "payment_type_id": range(1,len(payment_type_master)+1),
        "payment_type": payment_type_master

    }
) 

taxi_trips_payment_type_only = pd.DataFrame({
    "payment_type_id":[1,2,3],
    "payment_type": ["X","Y","Credit Card"]
})

taxi_trips_payment_type_only

,payment_type_id,payment_type
0,1,X
1,2,Y
2,3,Credit Card


In [19]:
def update_payment_type_master(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame)->pd.DataFrame:
    """Extend the payment type master with new payment if there are new payment types.

    Args:
        taxi_trips (pd.DataFrame): 
        DataFrame holding the daily taxi trips.
        company_master (pd.DataFrame): 
        DataFrame holding the payment_type data.

    Returns:
        pd.DataFrame: 
        The updated payment_type_master data, if new payment types are in the taxi data, they will be loaded to it.
    """

    payment_type_max_id = payment_type_master["payment_type_id"].max()

    new_payment_types_list= [payment_type for payment_type in taxi_trips["payment_type"].values if payment_type not in payment_type_master["payment_type"].values]
    new_payment_type_df = pd.DataFrame({
        "payment_type_id": range(payment_type_max_id +1, payment_type_max_id + len(new_payment_types_list)+1),
        "payment_type": new_payment_types_list
    })
        
    updated_payment_type_master = pd.concat([payment_type_master,new_payment_type_df], ignore_index=True)
    return updated_payment_type_master

In [20]:
updated_payment_type_master = update_payment_type_master(taxi_trips=taxi_trips_payment_type_only, payment_type_master=payment_type_master)

In [21]:
updated_payment_type_master.tail()

,payment_type_id,payment_type
3,4,Cash
4,5,Unknown
5,6,No Charge
6,7,X
7,8,Y


### Creating a generic update master table function


In [22]:
def update_master(taxi_trips: pd.DataFrame, master: pd.DataFrame, id_column: str, value_column: str)->pd.DataFrame:
    """Extend the  master DataFrame with new values nt if there are any new parameters.

    Args:
        taxi_trips (pd.DataFrame): 
        DataFrame holding the daily taxi trips.
        company_master (pd.DataFrame): 
        DataFrame holding the master data.
    id_column: str
        The id column of the master DataFrame.
    value_column: str
        Name of the column in master_df containing the values.

    Returns:
        pd.DataFrame: 
        The updated master data, if new values types are in the taxi data, they will be loaded to it.
    """

    max_id = master[id_column].max()

    new_values_list= [value for value in taxi_trips[value_column].values if value not in master[value_column].values]
    new_values_df = pd.DataFrame({
        id_column: range(max_id +1, max_id + len(new_values_list)+1),
        value_column: new_values_list
    })
        
    updated_master = pd.concat([master,new_values_df], ignore_index=True)
    return updated_master

In [23]:
test_payment_type_master = update_master(taxi_trips=taxi_trips_payment_type_only, master=payment_type_master, id_column="payment_type_id", value_column = "payment_type")

In [24]:
test_payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Mobile
2,3,Prcard
3,4,Cash
4,5,Unknown
5,6,No Charge
6,7,X
7,8,Y


In [25]:
test_company_mater = update_master(taxi_trips=taxi_trips_company_only, master=company_master, id_column="company_id", value_column = "company")

In [26]:
test_company_mater.tail()

,company_id,company
19,20,Tac - Checker Cab Dispatch
20,21,3591 - 63480 Chuks Cab
21,22,X
22,23,Y
23,24,Koam Taxi Association


### update taxi_trips with the most recent company master and payment_type master function

In [30]:
payment_type_master.tail()

,payment_type_id,payment_type
1,2,Mobile
2,3,Prcard
3,4,Cash
4,5,Unknown
5,6,No Charge


In [36]:
def update_taxi_trips_with_master_data(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame, company_master: pd.DataFrame)->pd.DataFrame:
    """Update the taxi_trips DataFrame with company_mater and payment_type_master ids, and delete string columns.

    Args:
        taxi_trips (pd.DataFrame): 
        The data with the daily taxi trips.

        payment_type_master (pd.DataFrame):
         The payment type master table.
        company_master (pd.DataFrame):
         The company master table.

    Returns:
        pd.DataFrame: 
        Taxi trips data, with only payment_type_id and company_id, without company or payment_type values.
    """

    taxi_trips_id = taxi_trips.merge(payment_type_master, on ="payment_type")
    taxi_trips_id = taxi_trips_id.merge(company_master, on ="company")

    taxi_trips_id.drop(["payment_type", "company"], axis =1, inplace=True)

    return taxi_trips_id

In [38]:
taxi_trips_id = update_taxi_trips_with_master_data(taxi_trips=taxi_trips, payment_type_master=payment_type_master, company_master=company_master)
taxi_trips_id.sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_community_area_id,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
0,4914bc18854fa2e923cbc848c41f64edf913b52c,e88fa53c774c82956a2aad60dcbdd260f05b157cf197da...,2024-03-13T23:45:00.000,2024-03-14T00:00:00.000,1130,17.23,76,42,9.3,0,4,55.8,41.980264315,-87.913624596,8,41.899602111,-87.633308037,2024-03-13 23:00:00,1,1
841,0273e2d7a6ab383a0960a191b62b33db9624b65b,ac488e1f03251055f4b4eba3ca51de23709ac4182721e0...,2024-03-13T21:45:00.000,2024-03-13T22:00:00.000,1128,8.24,76,22.75,5.45,0,4,32.7,41.980264315,-87.913624596,12,41.993930128,-87.758353588,2024-03-13 21:00:00,1,7
740,131897c0140133b4f3bd6aa86e87d030f395c964,d7a7bb002b49257d43e0f0c00702f1182ba37f9eda065c...,2024-03-13T22:00:00.000,2024-03-13T22:15:00.000,1436,17.47,76,42.5,5,0,4,52,41.97907082,-87.903039661,28,41.885300022,-87.642808466,2024-03-13 22:00:00,1,7
22,3b1144814ee2674f2cfd7ca25cc6579a888f3062,2981edd199f55bdc8b5ed188de69e9f5d8116ae8d7a434...,2024-03-13T23:45:00.000,2024-03-13T23:45:00.000,0,0,51,3.25,0,0,0,3.25,41.690633347,-87.570058269,51,41.690633347,-87.570058269,2024-03-13 23:00:00,4,4
100,b43d94b3237c67b5e14ba7ca30b5c9bb327892d4,a62c69900a839836b3a663a20124f35e40a4b825bf0dae...,2024-03-13T23:30:00.000,2024-03-14T00:00:00.000,1352,14.09,56,35.25,8.25,0,5.5,49.5,41.79259236,-87.769615453,7,41.922686284,-87.649488729,2024-03-13 23:00:00,1,3


### weather transformation function

In [39]:
def transform_weather_data(weather_data:json)->pd.DataFrame:
    """Make transformations on the daily weateher api resonpse.

    Args:
        weather_data (json): The daily weather data from the Opne Meteo API.

    Returns:
        pd.DataFrame: A DataFrame representation of the data.
    """
    #Transform part:
    weather_data_filtered = {
    "datetime": weather_data["hourly"]["time"],
    "temperature": weather_data["hourly"]["temperature_2m"],
    "windspeed": weather_data["hourly"]["wind_speed_10m"],
    "rain": weather_data["hourly"]["rain"],
    "precipitation": weather_data["hourly"]["precipitation"],

    }

    weather_df = pd.DataFrame(weather_data_filtered)
    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"])
    return weather_df

In [40]:
#extract
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d") 

url = "https://api.open-meteo.com/v1/forecast"
date = "2024-04-12"

params = {
    "latitude":41.85,
    "longitude": -87.65,
    "start_date": formatted_datetime,
    "end_date": formatted_datetime,
    "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
}
response = requests.get(url, params=params)
weather_data = response.json()
weather_data_df = transform_weather_data(weather_data)

In [41]:
weather_data_df.head()

,datetime,temperature,windspeed,rain,precipitation
0,2024-03-13 00:00:00,18.4,15.7,0.0,0.0
1,2024-03-13 01:00:00,17.0,9.8,0.0,0.0
2,2024-03-13 02:00:00,16.0,9.5,0.0,0.0
3,2024-03-13 03:00:00,14.6,7.9,0.0,0.0
4,2024-03-13 04:00:00,13.8,4.6,0.0,0.0
